## Back Test Data Generation

In [4]:
import requests
import time
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import pandas as pd
import numpy as np
import yaml
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from pycaret.classification import *

In [5]:
now = datetime.now()
 = now.strftime("%d-%m-%Y_%I-%M_%p")

In [6]:
with open ('back_test_pipeline_settings.yaml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    k_number = cfg['knn']['k_number']
    metric = cfg['knn']['metric']
    algorithm = cfg['knn']['algorithm']
    feature_1 = cfg['knn']['feature_1']
    feature_2 = cfg['knn']['feature_2']
    feature_3 = cfg['knn']['feature_3']
    feature_7 = cfg['knn']['feature_7']
    feature_8 = cfg['knn']['feature_8']   
    volume = cfg['feature']['volume']
    volume_size = cfg['sample']['volume_size']
    sample_count = cfg['sample']['count']
    candles = cfg['recommendation']['candle_count']
    pair = cfg['currency']['pair_1']

In [7]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [8]:
def find_k_similar_candles(candle_id, dataset, k=k_number):
    indices=[]
    distances = []
    output = []
    model_knn = NearestNeighbors(metric = metric, algorithm = algorithm) 
    model_knn.fit(dataset)
    
    #metric = 'euclidean' or 'cosine' or 'manhattan' or 'mahalanobis'
    
    distances, indices = model_knn.kneighbors(dataset.iloc[candle_id,:].values.reshape(1,-1),
                                              n_neighbors = k)

    for i in range(0,len(distances.flatten())):
        if i!=0:
            
            output.append ([dataset.index[indices.flatten()[i]],
                            distances.flatten()[i],
                            dataset.iloc[indices.flatten()[i]][feature_1],
                            dataset.iloc[indices.flatten()[i]][feature_2],
                            dataset.iloc[indices.flatten()[i]][feature_3],
                            dataset.iloc[indices.flatten()[i]][feature_7],
                            dataset.iloc[indices.flatten()[i]][feature_8],
                           ])
    
    output = pd.DataFrame(output)
    output.columns = ['Indice','Distance',
                      feature_1,
                      feature_2,
                      feature_3,
                      feature_7,
                      feature_8,
                     ]
   # display (output)
    
    return indices, distances

# <font color='red'>Test Configs</font>

In [9]:
filename = 'EUR_USD_H4.csv'
data = pd.read_csv(filename)

In [10]:
data.columns

Index(['Date', 'Time', 'f_time', 'julian_date', 'Weekday', 'Weekday_Name',
       'UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close', 'SMA_5', 'SMA_10',
       'SMA_20', 'F_SMA_5', 'F_SMA_10', 'F_SMA_20', 'O-H', 'O-L', 'O-C', 'H-L',
       'H-C', 'L-C', 'Direction', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5'],
      dtype='object')

In [11]:
data.head()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,SMA_5,SMA_10,SMA_20,F_SMA_5,F_SMA_10,F_SMA_20,O-H,O-L,O-C,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4,col_5
0,2015-10-01,01:00:00,1,274,3,Thursday,2015-10-01T01:00:00.000000000Z,1954,1.11727,1.11738,1.11446,1.11499,1.1175,1.1212,1.1213,-0.00251,-0.00621,-0.00631,-0.00011,0.00281,0.00228,0.00292,0.00239,-0.00053,0,0.00228,0.00011,0.00053,0.00025,0.00054
1,2015-10-01,05:00:00,5,274,3,Thursday,2015-10-01T05:00:00.000000000Z,5664,1.11498,1.11678,1.11350,1.11635,1.1169,1.1202,1.1211,-0.00055,-0.00385,-0.00475,-0.00180,0.00148,-0.00137,0.00328,0.00043,-0.00285,1,-0.00137,0.00180,0.00285,0.00228,0.00025
2,2015-10-01,09:00:00,9,274,3,Thursday,2015-10-01T09:00:00.000000000Z,6901,1.11632,1.11706,1.11441,1.11624,1.1165,1.1193,1.1210,-0.00026,-0.00306,-0.00476,-0.00074,0.00191,0.00008,0.00265,0.00082,-0.00183,0,0.00008,0.00074,0.00183,-0.00137,0.00228
3,2015-10-01,13:00:00,13,274,3,Thursday,2015-10-01T13:00:00.000000000Z,8686,1.11626,1.12092,1.11572,1.11964,1.1169,1.1188,1.1211,0.00274,0.00084,-0.00146,-0.00466,0.00054,-0.00338,0.00520,0.00128,-0.00392,1,-0.00338,0.00466,0.00392,0.00008,-0.00137
4,2015-10-01,17:00:00,17,274,3,Thursday,2015-10-01T17:00:00.000000000Z,3265,1.11960,1.12011,1.11808,1.11942,1.1173,1.1182,1.1213,0.00212,0.00122,-0.00188,-0.00051,0.00152,0.00018,0.00203,0.00069,-0.00134,0,0.00018,0.00051,0.00134,-0.00338,0.00008


## Selecting 1000 Random Candles

In [12]:
random_samples = data[data[volume] > volume_size].sample(n = sample_count)
Test_Candle = list(random_samples.index.values)

In [13]:
Test_Candle[0:10]

[4226, 4362, 2878, 2444, 2879, 1891, 8351, 566, 4340, 3153]

In [14]:
#Test_Candle = np.random.randint(low=1, high=len(data)-40, size=500)


In [15]:
data.shape

(9639, 30)

In [16]:
data.head(2)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,SMA_5,SMA_10,SMA_20,F_SMA_5,F_SMA_10,F_SMA_20,O-H,O-L,O-C,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4,col_5
0,2015-10-01,01:00:00,1,274,3,Thursday,2015-10-01T01:00:00.000000000Z,1954,1.11727,1.11738,1.11446,1.11499,1.1175,1.1212,1.1213,-0.00251,-0.00621,-0.00631,-0.00011,0.00281,0.00228,0.00292,0.00239,-0.00053,0,0.00228,0.00011,0.00053,0.00025,0.00054
1,2015-10-01,05:00:00,5,274,3,Thursday,2015-10-01T05:00:00.000000000Z,5664,1.11498,1.11678,1.11350,1.11635,1.1169,1.1202,1.1211,-0.00055,-0.00385,-0.00475,-0.00180,0.00148,-0.00137,0.00328,0.00043,-0.00285,1,-0.00137,0.00180,0.00285,0.00228,0.00025


# <font color='red'>CANDLE LOOP</font>

In [18]:
#LR_FINAL_MODEL = load_model('FINAL_MODELS/EURUSD/FINAL_LR_25Nov2021_EURUSD')
#ET_FINAL_MODEL = load_model('FINAL_MODELS/EURUSD/FINAL_ET_25Nov2021_EURUSD')
KNN_FINAL_MODEL = load_model('FINAL_MODELS/EURUSD/14-12-2021_12-59_AM_knn_EURUSD')
#LIGHTGBM_FINAL_MODEL = load_model('FINAL_MODELS/EURUSD/01-12-2021_08-02_PM_lightgbm_EURUSD')
GBC_FINAL_MODEL = load_model('FINAL_MODELS/EURUSD/14-12-2021_09-08_AM_gbc_EURUSD')

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


In [19]:
%%time
print ('Today: ' + today)

result_output = pd.DataFrame({'Candle_No':[],
                              'Current_Market_Fit':[],
                              'Current_Market':[],
#                              'Rec1':[],
#                              'Rec1_P':[],
#                              'Rec2':[],
#                              'Rec2_P':[],
#                              'Rec3':[],
#                              'Rec3_P':[],
 #                             'LR_Label':[],
 #                             'LR_Score':[],
 #                             'ET_Label':[],
 #                             'ET_Score':[],
                              'KNN_Label':[],
                              'KNN_Score':[],
 #                             'LIGHTGBM_Label':[],
 #                             'LIGHTGBM_Score':[],
                              'GBC_Label':[],
                              'GBC_Score':[],                              
                             })

for candle_no in Test_Candle:
    data = pd.read_csv(filename)
    data = data.iloc[candle_no:candle_no+candles]
    data['candleno'] = range (1, len(data) + 1)
    X = data['candleno'].values.reshape(-1, 1)
    Y = data['Close'].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)
    y_pred = linear_regressor.predict(X) 
    
    Current_Market_Fit = r2_score(Y, y_pred).round(2)*100
    coeficient = (linear_regressor.coef_)

    if coeficient > 0:
        Current_Market= 1

    else:
        Current_Market = 0

    data = pd.read_csv(filename)
    data = data[[feature_1,
                 feature_2,
                 feature_3,
                 feature_7,
                 feature_8,
                ]]

    indices, distances = find_k_similar_candles (candle_no,data)
    indices = indices[0:1][0]
    
    predicted_output = []
    recs = []
    for indice in indices[1:5]:
             
        Predicted_Market_Fit =0
        Predicted_Trade=''
    
        data = pd.read_csv(filename) 
        data = data.iloc[indice:indice+candles]

        data['candleno'] = range (1, len(data) + 1)
        X = data['candleno'].values.reshape(-1, 1)
        Y = data['Close'].values.reshape(-1, 1)
        linear_regressor = LinearRegression()
        linear_regressor.fit(X, Y)
        y_pred = linear_regressor.predict(X)

        Predicted_Market_Fit= r2_score(Y, y_pred)*100
        coeficient = (linear_regressor.coef_)

        if coeficient > 0:
            Predicted_Trade = 'BUY'
            recs.append((r2_score(Y, y_pred)*100))
        else:
            Predicted_Trade = 'SELL'
            recs.append((r2_score(Y, y_pred)*100) * -1)
        
        predicted_output.append([Predicted_Market_Fit,Predicted_Trade])
        
        
    data_unseen = pd.DataFrame ({
        'Rec1_Score': [recs[0]],
        'Rec2_Score': [recs[1]],
        'Rec3_Score': [recs[2]],
        'Rec4_Score': [recs[3]],
    })
    
#    lr_prediction = predict_model(LR_FINAL_MODEL, data=data_unseen)
#    LR_Label = lr_prediction['Label']
#    LR_Score = lr_prediction['Score']
    
#    et_prediction = predict_model(ET_FINAL_MODEL, data=data_unseen)
#    ET_Label = et_prediction['Label']
#    ET_Score = et_prediction['Score']
    
    knn_prediction = predict_model(KNN_FINAL_MODEL, data=data_unseen)
    KNN_Label = knn_prediction['Label']
    KNN_Score = knn_prediction['Score']
    
#    lightgbm_prediction = predict_model(LIGHTGBM_FINAL_MODEL, data=data_unseen)
#    LIGHTGBM_Label = lightgbm_prediction['Label']
#    LIGHTGBM_Score = lightgbm_prediction['Score']

    gbc_prediction = predict_model(GBC_FINAL_MODEL, data=data_unseen)
    GBC_Label = gbc_prediction['Label']
    GBC_Score = gbc_prediction['Score']    
    
    result = {'Candle_No': candle_no,
              'Current_Market_Fit': Current_Market_Fit,
              'Current_Market': Current_Market,
#              'Rec1': predicted_output[0][0],
#              'Rec1_P': predicted_output[0][1],
#              'Rec2': predicted_output[1][0],
#              'Rec2_P': predicted_output[1][1],
#              'Rec3': predicted_output[2][0],
#              'Rec3_P': predicted_output[2][1],
  #            'LR_Label': LR_Label[0],
  #            'LR_Score': LR_Score[0],
  #            'ET_Label': ET_Label[0],
  #            'ET_Score': ET_Score[0],
              'KNN_Label': KNN_Label[0],
              'KNN_Score': KNN_Score[0],
  #            'LIGHTGBM_Label': LIGHTGBM_Label[0],
  #            'LIGHTGBM_Score': LIGHTGBM_Score[0],
              'GBC_Label': GBC_Label[0],
              'GBC_Score': GBC_Score[0],              
             }
    
    result_output = result_output.append(result, ignore_index = True)

Today: 14-12-2021_09-35_AM
CPU times: user 1h 2min 34s, sys: 27min 58s, total: 1h 30min 33s
Wall time: 42min 36s


In [21]:
result_output.to_csv('03_Back_Test_Final_Result_' + today + '.csv', header = True, index = False)
result_output = pd.read_csv('03_Back_Test_Final_Result_' + today + '.csv')

In [22]:
result_output = result_output[result_output['Current_Market_Fit'] > 20]
result_output = result_output.reset_index()
del result_output['index']

In [23]:
result_output.shape

(2868, 7)

In [24]:
result_output.to_csv('03_Back_Test_Final_Result_' + today + '.csv', header = True, index = False)
result_output = pd.read_csv('03_Back_Test_Final_Result_' + today + '.csv')

In [25]:
#result_output['LR_Prediction'] = result_output['Current_Market'] - result_output['LR_Label']
#result_output['ET_Prediction'] = result_output['Current_Market'] - result_output['ET_Label']
result_output['KNN_Prediction'] = result_output['Current_Market'] - result_output['KNN_Label']
#result_output['LIGHTGBM_Prediction'] = result_output['Current_Market'] - result_output['LIGHTGBM_Label']
result_output['GBC_Prediction'] = result_output['Current_Market'] - result_output['GBC_Label']

In [26]:
result_output.head()

,Candle_No,Current_Market_Fit,Current_Market,KNN_Label,KNN_Score,GBC_Label,GBC_Score,KNN_Prediction,GBC_Prediction
0,4226.0,62.0,1.0,0.0,0.5129,1.0,0.5979,1.0,0.0
1,2879.0,58.0,1.0,1.0,0.9991,1.0,0.8439,0.0,0.0
2,1891.0,84.0,0.0,0.0,0.9982,0.0,0.9731,0.0,0.0
3,8351.0,79.0,0.0,0.0,0.9977,0.0,0.8564,0.0,0.0
4,566.0,80.0,0.0,0.0,0.9969,0.0,0.7135,0.0,0.0


In [27]:
KNN_Result = pd.DataFrame(result_output['KNN_Prediction'].value_counts())
KNN_Result["Score"] = (KNN_Result['KNN_Prediction'] * 100 / len(result_output)).round(2)

In [28]:
GBC_Result = pd.DataFrame(result_output['GBC_Prediction'].value_counts())
GBC_Result["Score"] = (GBC_Result['GBC_Prediction'] * 100 / len(result_output)).round(2)

In [29]:
result = pd.concat([
#    LR_Result,
#    ET_Result,
    KNN_Result,
    GBC_Result,
], axis=1)
result

,KNN_Prediction,Score,GBC_Prediction,Score
-1.0,283,9.87,346,12.06
0.0,2301,80.23,2210,77.06
1.0,284,9.90,312,10.88
